In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from methods import load_data, NeuralNet, loader_eval, load_data_with_validation

c:\Users\ezer6\anaconda3\envs\adl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
def train_model(model, train_loader, test_loader, val_loader, num_epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    train_loss_per_epoch = []
    test_loss_per_epoch = []
    val_loss_per_epoch = []
    for epoch in range(num_epochs):
        # Train
        train_loss = 0
        for i, (images, labels) in enumerate(train_loader):
            images = images.reshape(-1, input_size).to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_loss_per_epoch.append(train_loss)
        
        # Validate
        val_loss = loader_eval(val_loader, model, criterion)
        val_loss_per_epoch.append(val_loss)

        # Test
        test_loss = loader_eval(test_loader, model, criterion)
        test_loss_per_epoch.append(test_loss)
        
    train_loss_per_epoch = np.array(train_loss_per_epoch)
    test_loss_per_epoch = np.array(test_loss_per_epoch)
    val_loss_per_epoch = np.array(val_loss_per_epoch)
    
    return train_loss_per_epoch, test_loss_per_epoch, val_loss_per_epoch  

In [4]:
def grid_search(hidden_sizes=[hidden_size], batch_sizes=[batch_size], learning_rates=[learning_rate], num_epochs=5):
    
    paramaters_loss = {}
    iter = 0

    for b_size in batch_sizes:
            train_loader, test_loader, val_loader = load_data_with_validation(batch_size=b_size)
            
            for h_size in hidden_sizes:
                
                for l_rate in learning_rates:
                    model = NeuralNet(input_size, hidden_size, num_classes).to(device)
                    paramaters_loss[(b_size, h_size, l_rate)] = train_model(model, train_loader, test_loader,
                                                                                        val_loader)[1:] #only val,test loss
                    iter +=1
                    print(f'iterations : {iter}')
    return paramaters_loss

In [5]:
hidden_sizes = [400, 500, 600]
batch_sizes = [100, 200, 300]
learning_rates = [0.01, 0.001]

In [6]:
parameters_loss = grid_search(hidden_sizes, batch_sizes, learning_rates)

iterations : 1
iterations : 2
iterations : 3
iterations : 4
iterations : 5
iterations : 6
iterations : 7
iterations : 8
iterations : 9
iterations : 10
iterations : 11
iterations : 12
iterations : 13
iterations : 14
iterations : 15
iterations : 16
iterations : 17
iterations : 18


In [7]:
def min_validation_loss(test_loss, val_loss):
    min_index = np.argmin(val_loss)
    return test_loss[min_index], val_loss[min_index]

In [8]:
# create csv file with errors for each combination
import csv 

s_grid = []
fields = ['Batch Size', 'Hidden Size', 'Learning Rate', 'Best validation loss', 'Test error for same epoch']
for key in parameters_loss.keys():
    b_size, h_size, l_rate = key
    test_loss, val_loss = parameters_loss[key]
    test_loss, val_loss = min_validation_loss(test_loss, val_loss)
    s_grid.append({'Batch Size':str(b_size),
                   'Hidden Size':str(h_size),
                   'Learning Rate': str(l_rate),
                   'Best validation loss': str(val_loss),
                   'Test error for same epoch':str(test_loss)})

# writing to the CSV file
with open('search grid.csv', 'w', newline='') as csvfile:
    # creating a csv dict writer object
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    
    # writing headers (field names)
    writer.writeheader()
    
    # writing rows (dictionary values)
    for row in s_grid:
        writer.writerow(row)